In [127]:
import pandas as pd
import numpy as np
from vali_tools.valitools import check_health
pd.set_option('display.max_column',None)
import datetime

In [128]:
cyc = pd.read_csv(r"Source\Data\cycles.csv")
far = pd.read_csv(r"Source\Data\farms.csv")
fas = pd.read_csv(r"Source\Data\fasting.csv")
fas_s = pd.read_csv(r"Source\Data\fastings.csv")
fet = pd.read_csv(r"Source\Data\feed_tray.csv")
fee = pd.read_csv(r"Source\Data\feeds.csv")
har = pd.read_csv(r"Source\Data\harvests.csv")
mea = pd.read_csv(r"Source\Data\measurements.csv")
mor = pd.read_csv(r"Source\Data\mortalities.csv")
pon = pd.read_csv(r"Source\Data\ponds.csv")
sam = pd.read_csv(r"Source\Data\samplings.csv")

In [131]:
cyc.sample(5)

,cycle_id,pond_id,species_id,total_seed,started_at,finished_at,remark,created_at,updated_at,area,initial_age,limit_weight_per_area,target_cultivation_day,target_size,extracted_at,subscription_type,ordered_at,hatchery_id,total_seed_type,hatchery_name,pond_name,pond_length,pond_width,pond_depth,cyc_dur
2393,11341,23677,1.0,25000,2021-08-09,2021-11-16,NaN,2021-08-10 08:24:46,2021-12-06 05:10:10,120.00,0.0,1.25,128.0,45.0,2024-04-12 17:02:22,Free,2021-08-20 06:05:21,NaN,gross,NaN,WR2,12.65,9.49,1.0,99
772,15015,31256,1.0,270900,2022-03-30,2022-06-10,NaN,2022-03-19 05:24:15,2022-06-17 00:09:19,1881.00,0.0,1.25,105.0,40.0,2024-04-12 17:02:22,Free,2022-03-19 08:27:55,1007.0,net,CV. Mitra Larva Sejahtera,C1,49.50,38.00,1.2,72
559,21485,36993,1.0,110000,2022-10-25,2023-02-22,#JT03,2023-03-31 02:39:30,2023-03-31 02:58:41,1200.00,0.0,1.25,120.0,55.0,2024-04-12 17:02:22,Free,NaT,26.0,net,PT. Suri Tani Pemuka Unit Hatchery Indramayu,A1,40.00,30.00,1.0,120
1499,20384,34736,NaN,294252,2023-01-12,2023-01-17,NaN,2023-01-12 10:36:03,2023-01-21 06:43:39,35.75,0.0,1.25,5.0,55.0,2024-04-12 17:02:22,Free,NaT,1020.0,actual,MATERNIDADE NACALA,N03,7.00,5.00,1.0,5
439,18324,33007,1.0,125000,2022-09-15,2022-11-05,#SiklusTebar,2022-09-15 15:48:59,2022-11-08 15:09:55,750.00,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,2022-09-15 15:48:59,77.0,net,Suri Tani Pemuka Anyer,A1,30.00,25.00,1.0,51


## Note Cycle
- id should be unique
- there are duplicate on id
- features: area, initial_age,species_id,subscription_type,cycle_dur(new)

In [130]:
# cyc treatment
cyc.id = cyc.id.astype(str) 
cyc.pond_id = cyc.pond_id.astype(str) 
cyc.species_id = cyc.species_id.astype(str)
cyc = cyc.drop_duplicates(subset='id')

cyc.started_at = pd.to_datetime(cyc.started_at)
cyc.finished_at = pd.to_datetime(cyc.finished_at)
cyc.created_at = pd.to_datetime(cyc.created_at)
cyc.updated_at = pd.to_datetime(cyc.updated_at)
cyc.extracted_at = pd.to_datetime(cyc.extracted_at)
cyc.ordered_at = pd.to_datetime(cyc.ordered_at)
cyc.finished_at.fillna(datetime.datetime.now(),inplace=True)

cyc.species_id = cyc.species_id.replace('nan',np.nan)

cyc['cyc_dur'] = cyc.finished_at - cyc.started_at
cyc['cyc_dur'] = cyc['cyc_dur'].astype(str)
cyc['cyc_dur'] = cyc['cyc_dur'].str.split(' ').str[0]
cyc['cyc_dur'] = cyc['cyc_dur'].astype(int)
cyc.rename(columns={'id':'cycle_id'},inplace=True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_37696\3077323768.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cyc.finished_at.fillna(datetime.datetime.now(),inplace=True)


In [132]:
check_health(cyc)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2500,2500,0,0.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,20849,1
1,pond_id,0,1675,2500,825,0.0000,0.6700,0.3300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,24569,8
2,species_id,790,2,2500,2498,0.3160,0.0008,0.9992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1.0,1576
3,total_seed,0,868,2500,1632,0.0000,0.3472,0.6528,217154.5784,170662.188169,10.0,1800000.0,90000.0,184722.0,300000.0,numeric,NaN,NaN
4,started_at,0,750,2500,1750,0.0000,0.3000,0.7000,2022-10-27 02:43:00.479999744,NaN,2020-03-07 00:00:00,2024-02-18 00:00:00,2022-05-08 00:00:00,2022-11-16 00:00:00,2023-08-05 00:00:00,numeric,NaN,NaN
5,finished_at,0,858,2500,1642,0.0000,0.3432,0.6568,2023-01-13 23:23:43.568200192,NaN,2020-05-20 00:00:00,2024-09-04 00:35:20.500818,2022-07-16 18:00:00,2023-02-10 00:00:00,2023-10-26 00:00:00,numeric,NaN,NaN
6,remark,1172,67,2500,2433,0.4688,0.0268,0.9732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,#SiklusFarm,269
7,created_at,0,2325,2500,175,0.0000,0.9300,0.0700,2022-11-10 21:50:26.408800,NaN,2020-02-19 08:44:53,2024-03-05 13:10:13,2022-05-17 04:53:08,2022-12-02 04:03:08,2023-08-10 03:05:55.750000128,numeric,NaN,NaN
8,updated_at,0,2289,2500,211,0.0000,0.9156,0.0844,2023-03-08 00:19:24.160000,NaN,2020-06-03 02:44:52,2024-04-09 00:06:26,2022-08-25 12:30:48.500000,2023-06-05 00:03:12,2023-11-20 07:32:10,numeric,NaN,NaN
9,area,0,648,2500,1852,0.0000,0.2592,0.7408,2233.092964,12372.846032,1.02,422500.0,803.84,1368.0,2238.6825,numeric,NaN,NaN


In [22]:
far.sample(5)

,id,province,regency,timezone
331,9422,JAWA TIMUR,JEMBER,+07:00
128,6149,ACEH,PIDIE JAYA,+07:00
290,8686,NaN,NaN,+07:00
311,8933,BANTEN,NaN,+07:00
447,11180,JAWA TENGAH,KEBUMEN,+07:00


In [7]:
check_health(far)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,551,551,0,0.000000,1.000000,0.000000,8625.956443,2937.544875,652.0,13886.0,6369.5,8531.0,10710.0,numeric,NaN,NaN
1,province,72,20,551,531,0.130672,0.036298,0.963702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,JAWA TIMUR,167
2,regency,93,67,551,484,0.168784,0.121597,0.878403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,BANYUWANGI,87
3,timezone,0,6,551,545,0.000000,0.010889,0.989111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,+07:00,492


## Note fasting
- features: fasting

In [133]:
fast = pd.concat([fas,fas_s])

In [134]:
fast.logged_date = pd.to_datetime(fast.logged_date)
fast.cycle_id = fast.cycle_id.astype(str)
fast.fasting = fast.fasting.astype(str)
fast.cycle_id = fast.cycle_id.str.split('.').str[0]
fast.fasting = fast.fasting.str.split('.').str[0]
fast = fast.drop_duplicates(subset=['logged_date','cycle_id','fasting'])
fast.fasting.replace('nan','0',inplace=True)
fast.fasting = fast.fasting.astype(int)

fast_agg = fast.groupby(['cycle_id'],as_index=False).fasting.sum()

C:\Users\kevin\AppData\Local\Temp\ipykernel_37696\1786335512.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fast.fasting.replace('nan','0',inplace=True)


In [120]:
check_health(fast)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,logged_date,0,1154,38108,36954,0.0,0.030282,0.969718,2023-01-03 03:54:19.158181888,NaN,2018-06-08 00:00:00,2024-03-31 00:00:00,2022-08-28 00:00:00,2022-11-17 00:00:00,2023-07-10 00:00:00,numeric,NaN,NaN
1,cycle_id,0,1797,38108,36311,0.0,0.047155,0.952845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,15982,128
2,fasting,0,2,38108,38106,0.0,0.000052,0.999948,0.262779,0.440149,0.0,1.0,0.0,0.0,1.0,numeric,NaN,NaN


In [135]:
cyc.merge(fast_agg,how='left',on='cycle_id')

,cycle_id,pond_id,species_id,total_seed,started_at,finished_at,remark,created_at,updated_at,area,initial_age,limit_weight_per_area,target_cultivation_day,target_size,extracted_at,subscription_type,ordered_at,hatchery_id,total_seed_type,hatchery_name,pond_name,pond_length,pond_width,pond_depth,cyc_dur,fasting
0,18876,36294,1.0,331920,2022-10-14,2023-01-29,NaN,2022-10-14 12:34:23,2023-01-30 01:46:27,4030.0,0.0,2.00,110.0,35.0,2024-04-12 17:02:22,Free,NaT,66.0,net,UD. Benur Ndaru Laut,H,65.00,62.00,1.0,107,22.0
1,22118,37102,1.0,40000,2023-04-26,2023-06-23,#TBR01,2023-04-30 22:28:13,2023-06-26 05:35:32,399.0,0.0,1.25,120.0,100.0,2024-04-12 17:02:22,Free,NaT,83.0,net,CV Raja Benur,A1,30.00,20.00,1.0,58,1.0
2,24088,39154,NaN,357570,2023-08-01,2023-10-18,#SiklusFarm,2023-07-12 01:13:05,2023-10-19 04:23:11,2000.0,0.0,1.25,120.0,20.0,2024-04-12 17:02:22,Free,2023-07-12 01:13:05,1004.0,gross,PT. Tri Karta Pratama - Total Kualitas Prima,A3,50.00,40.00,1.4,78,2.0
3,17743,20914,1.0,168275,2022-07-19,2022-09-29,#SiklusCustomerPermadi,2022-08-14 04:40:46,2022-10-06 00:09:36,1512.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,NaT,3.0,actual,PT. Central Proteinaprima Tbk,B4,54.00,28.00,1.5,72,0.0
4,17125,34187,1.0,188000,2022-07-07,2022-09-20,#SiklusTebar,2022-07-12 00:24:30,2022-10-15 04:27:24,1225.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,2022-07-12 00:24:30,35.0,net,PT Windu Alam Sentosa,A,35.00,35.00,1.2,75,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,19131,34712,2.0,300000,2022-10-25,2023-01-21,NaN,2022-10-25 10:13:31,2023-01-25 08:01:19,4999.9,0.0,1.25,88.0,55.0,2024-04-12 17:02:22,Free,NaT,1020.0,actual,MATERNIDADE NACALA,E01,70.71,70.71,1.5,88,0.0
2496,27552,45697,1.0,100000,2023-11-05,2024-02-13,#TBR03,2023-11-18 03:43:40,2024-03-27 02:52:36,900.0,0.0,1.25,120.0,55.0,2024-04-12 17:02:22,Free,2023-11-18 03:43:40,1022.0,net,Windu Segara Anyar,Kolam 4,45.00,20.00,1.5,100,14.0
2497,27885,24570,1.0,30716,2023-12-02,2024-02-29,#TBR05,2023-12-03 03:06:01,2024-03-03 03:03:26,86.0,0.0,1.25,120.0,20.0,2024-04-12 17:02:22,Free,NaT,83.0,gross,CV Raja Benur,Bagus,10.71,8.03,1.0,89,3.0
2498,15868,32398,1.0,150000,2022-04-28,2022-06-21,#SiklusTebar,2022-05-08 16:02:51,2022-08-23 03:12:20,1600.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,2022-05-08 16:02:51,35.0,gross,PT Windu Alam Sentosa,B 4.1,45.00,35.00,2.0,54,2.0


In [11]:
fet.sample(5)

,id,logged_at,feed_logged_at,cycle_id,tray_number,feed_remaining_percent,remark,created_at,updated_at,local_feed_logged_at
152796,194131,2023-10-29 14:00:00,2023-10-29 12:00:00,23710,1,66,"0,8% pakan",2023-10-29 11:41:36,2023-10-29 15:34:50,2023-10-29 21:00:00
73766,98486,2022-11-06 05:00:00,2022-11-06 03:00:00,18615,1,0,"Pakan di anco 0,5%",2022-11-08 03:55:03,2022-11-08 03:55:03,2022-11-06 12:00:00
142754,183148,2023-09-16 01:30:00,2023-09-15 23:30:00,23734,1,0,NaN,2023-09-18 02:52:13,2023-09-18 02:52:13,2023-09-16 08:30:00
51392,70127,2022-08-19 09:00:00,2022-08-19 08:00:00,16712,2,0,NaN,2022-08-19 10:11:13,2022-08-19 10:11:13,2022-08-19 16:00:00
132739,172336,2023-08-02 06:30:00,2023-08-02 03:30:00,23734,2,0,NaN,2023-08-02 11:16:53,2023-08-02 11:16:53,2023-08-02 13:30:00


In [12]:
check_health(fet)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,186664,186664,0,0.000000,1.000000,0.000000,123263.899429,67748.121524,498.0,285950.0,63660.75,122951.5,180322.25,numeric,NaN,NaN
1,logged_at,0,16592,186664,170072,0.000000,0.088887,0.911113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-06-12 00:30:00,125
2,feed_logged_at,0,15298,186664,171366,0.000000,0.081955,0.918045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-06-23 00:00:00,224
3,cycle_id,0,709,186664,185955,0.000000,0.003798,0.996202,20091.738193,4137.425172,12163.0,29874.0,16162.00,19253.0,23739.00,numeric,NaN,NaN
4,tray_number,0,6,186664,186658,0.000000,0.000032,0.999968,1.950221,1.077650,1.0,6.0,1.00,2.0,3.00,numeric,NaN,NaN
5,feed_remaining_percent,0,3,186664,186661,0.000000,0.000016,0.999984,10.079935,19.511848,0.0,66.0,0.00,0.0,0.00,numeric,NaN,NaN
6,remark,140283,739,186664,185925,0.751527,0.003959,0.996041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1% DARI PAKAN,2849
7,created_at,0,38870,186664,147794,0.000000,0.208235,0.791765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-05-17 14:52:36,120
8,updated_at,0,38949,186664,147715,0.000000,0.208658,0.791342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-05-18 13:27:06,120
9,local_feed_logged_at,0,16510,186664,170154,0.000000,0.088448,0.911552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-07-08 07:30:00,124


In [13]:
fee.sample(5)

,cycle_id,logged_at,quantity
48070,24304,2023-10-23 05:00:00.000,8.0
248523,18253,2022-11-07 09:55:00.000,10.0
604693,20887,2023-03-26 08:00:00.000,3.1
361300,16051,2022-07-26 05:30:00.000,11.0
674390,27092,2023-10-26 10:00:30.000,30.0


In [14]:
check_health(fee)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2500,706908,704408,0.00000,0.003537,0.996463,21890.928919,5434.180295,3458.0,29874.0,18395.0,23633.0,27089.0,numeric,NaN,NaN
1,logged_at,21,74841,706908,632067,0.00003,0.105871,0.894129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-12 23:00:00.000,576
2,quantity,21,6404,706908,700504,0.00003,0.009059,0.990941,17.064466,22.536168,-1.0,2011.0,4.0,10.0,22.0,numeric,NaN,NaN


In [15]:
har.sample(5)

,cycle_id,updated_at,size,created_at,weight,id,harvested_at,status,selling_price
5805,16039.0,2022-08-19 04:30:01,48.00,2022-08-19 04:30:01,778.45,12617.0,2022-08-18,Full,NaN
4966,17743.0,2022-10-03 06:47:10,75.88,2022-10-03 06:47:10,58.28,13481.0,2022-09-29,Full,NaN
1649,9482.0,2021-06-27 08:57:50,200.00,2021-06-27 08:57:03,120.00,5714.0,2021-06-27,Full,120.0
7214,20531.0,2023-03-18 13:26:05,128.00,2023-03-18 13:26:05,10.90,16816.0,2023-03-18,Partial,654000.0
5437,18542.0,2022-12-22 07:40:24,230.00,2022-12-22 07:40:24,6.00,15297.0,2022-12-04,Failed,312000.0


In [16]:
check_health(har)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2490,8087,5597,0.000000,0.307902,0.692098,1.987177e+04,5.785357e+03,3458.0,2.987400e+04,16342.50,19672.0,2.474250e+04,numeric,NaN,NaN
1,updated_at,0,5892,8087,2195,0.000000,0.728577,0.271423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2021-08-03 13:28:11,63
2,size,0,1481,8087,6606,0.000000,0.183133,0.816867,1.138034e+02,3.811497e+02,10.0,2.000000e+04,60.00,86.0,1.250000e+02,numeric,NaN,NaN
3,created_at,0,7098,8087,989,0.000000,0.877705,0.122295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2021-07-12 12:53:37,40
4,weight,0,5109,8087,2978,0.000000,0.631755,0.368245,7.746361e+02,4.266748e+03,0.0,2.500000e+05,47.97,250.0,6.676950e+02,numeric,NaN,NaN
5,id,0,8076,8087,11,0.000000,0.998640,0.001360,1.730768e+04,6.714158e+03,2052.0,2.846800e+04,12549.50,16555.0,2.311150e+04,numeric,NaN,NaN
6,harvested_at,0,1069,8087,7018,0.000000,0.132187,0.867813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-10-15,73
7,status,263,13,8087,8074,0.032521,0.001608,0.998392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,Partial,4192
8,selling_price,1793,4981,8087,3106,0.221714,0.615927,0.384073,3.181221e+07,1.646234e+08,0.0,1.125000e+10,298936.80,6247500.0,2.589123e+07,numeric,NaN,NaN


In [17]:
mea.sample(5)

,pond_id,cycle_id,measured_date,morning_temperature,evening_temperature,morning_do,evening_do,morning_salinity,evening_salinity,morning_pH,evening_pH,transparency,turbidity,ammonia,nitrate,nitrite,alkalinity,hardness,calcium,magnesium,carbonate,bicarbonate,tom,total_plankton_
47532,39087,21203,2023-04-25,28.65,NaN,2.48,NaN,19.35,NaN,8.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51595,44540,29100,2023-08-21,32.00,NaN,4.60,NaN,36.00,NaN,9.00,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71040,46258,28067,2024-01-05,28.60,29.5,5.30,5.5,28.00,28.0,7.72,7.75,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136926,40661,25307,2023-09-15,NaN,NaN,NaN,NaN,NaN,NaN,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134682,45257,27088,2023-10-18,NaN,NaN,NaN,NaN,36.00,NaN,7.90,8.20,25.0,NaN,NaN,10.0,0.0,176.0,NaN,NaN,NaN,0.0,176.0,83.0,3290000.0


In [18]:
check_health(mea)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,pond_id,0,1675,139050,137375,0.000000,0.012046,0.987954,32589.530004,1.022891e+04,1.000,4.728200e+04,30606.0000,34706.000,39152.000,numeric,NaN,NaN
1,cycle_id,0,2500,139050,136550,0.000000,0.017979,0.982021,20268.853290,5.401437e+03,3458.000,2.987400e+04,16776.0000,20197.000,24739.000,numeric,NaN,NaN
2,measured_date,0,1427,139050,137623,0.000000,0.010262,0.989738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-11,343
3,morning_temperature,41855,4793,139050,134257,0.301007,0.034470,0.965530,28.580945,1.068968e+02,0.000,2.734100e+04,27.0000,28.000,29.000,numeric,NaN,NaN
4,evening_temperature,49839,5386,139050,133664,0.358425,0.038734,0.961266,29.908567,5.817013e+00,0.000,1.078933e+03,28.9000,30.000,31.000,numeric,NaN,NaN
5,morning_do,61324,6531,139050,132519,0.441021,0.046969,0.953031,5.904100,2.168104e+02,-52.885,6.004300e+04,4.2310,4.970,5.640,numeric,NaN,NaN
6,evening_do,65098,7706,139050,131344,0.468163,0.055419,0.944581,6.005278,3.044450e+01,-62.837,6.706000e+03,4.8310,5.640,6.500,numeric,NaN,NaN
7,morning_salinity,46388,19113,139050,119937,0.333607,0.137454,0.862546,22.777202,6.078861e+01,-25.000,1.573400e+04,16.8000,23.000,29.000,numeric,NaN,NaN
8,evening_salinity,73957,18407,139050,120643,0.531873,0.132377,0.867623,22.090321,1.526926e+02,0.000,3.395800e+04,15.6880,21.400,26.950,numeric,NaN,NaN
9,morning_pH,20115,3190,139050,135860,0.144660,0.022941,0.977059,8.006981,2.202525e+01,0.000,7.421000e+03,7.6150,7.900,8.120,numeric,NaN,NaN


In [19]:
mor.sample(5)

,id,cycle_id,quantity,recorded_at,created_at,updated_at,average_weight
1213,6619,15495,4695,2022-06-30,2022-07-02 07:50:45,2024-01-08 11:40:40,4.26
6048,13339,18675,72,2023-01-18,2023-01-25 15:08:17,2024-01-08 11:40:40,24.39
11094,20856,26245,60,2023-11-28,2023-12-06 08:07:46,2023-12-06 08:07:47,10.58
12624,23093,27729,140,2024-02-08,2024-02-14 18:26:02,2024-02-14 18:26:08,14.29
1916,7610,16469,1036,2022-07-23,2022-07-24 02:18:19,2024-01-08 11:40:40,1.93


In [20]:
check_health(mor)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,13221,13221,0,0.0,1.000000,0.000000,14307.947810,5630.215217,185.0,26192.00,9763.00,13998.00,18858.00,numeric,NaN,NaN
1,cycle_id,0,857,13221,12364,0.0,0.064821,0.935179,20844.158763,4098.897296,7273.0,29579.00,17726.00,19676.00,24092.00,numeric,NaN,NaN
2,quantity,0,2538,13221,10683,0.0,0.191967,0.808033,1834.642085,87210.179102,1.0,10000000.00,60.00,200.00,756.00,numeric,NaN,NaN
3,recorded_at,0,896,13221,12325,0.0,0.067771,0.932229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-13,55
4,created_at,0,11340,13221,1881,0.0,0.857726,0.142274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-01-24 04:36:05,27
5,updated_at,0,1475,13221,11746,0.0,0.111565,0.888435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2024-01-08 11:40:40,11713
6,average_weight,0,1984,13221,11237,0.0,0.150064,0.849936,9.613240,17.196472,0.0,1428.57,4.55,7.89,13.04,numeric,NaN,NaN


In [136]:
pon.sample(5)

,id,farm_id,length,width,deep,created_at,updated_at,record_id,extracted_at,max_seed_density
291,44606,12550,150.00,20.00,1.0,2023-10-13 07:50:24.000,2023-10-13 07:50:24.000,ponds_2024_4_12_44606,2024-04-12 17:02:11.000,NaN
184,26461,6738,31.41,23.56,NaN,2021-09-14 00:54:35.000,2021-09-14 00:54:35.000,ponds_2024_4_12_26461,2024-04-12 17:02:11.000,NaN
217,44564,12539,99.00,50.00,1.0,2023-10-11 12:45:43.000,2023-10-11 12:45:43.000,ponds_2024_4_12_44564,2024-04-12 17:02:11.000,101.010101
11,28868,7383,51.64,38.73,NaN,2021-11-30 08:59:31.000,2021-11-30 08:59:31.000,ponds_2024_4_12_28868,2024-04-12 17:02:11.000,50.000000
137,44616,12553,40.00,40.00,1.5,2023-10-14 03:18:54.000,2023-10-14 03:29:59.000,ponds_2024_4_12_44616,2024-04-12 17:02:11.000,NaN


In [137]:
far

,id,province,regency,timezone
0,652,JAWA BARAT,SUBANG,+07:00
1,869,SULAWESI SELATAN,BARRU,+08:00
2,900,JAWA TIMUR,PACITAN,+07:00
3,1000,JAWA TIMUR,SITUBONDO,+07:00
4,1106,JAWA TIMUR,BANYUWANGI,+07:00
...,...,...,...,...
546,13692,OUTSIDE INDONESIA,NaN,+07:00
547,13743,BALI,KARANG ASEM,+08:00
548,13788,JAWA TIMUR,JEMBER,+07:00
549,13876,JAWA TIMUR,BANYUWANGI,+07:00


In [22]:
check_health(pon)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,338,338,0,0.000000,1.000000,0.000000,36899.159763,12968.985879,1.000000,47762.00,28825.250000,44596.50,44702.750000,numeric,NaN,NaN
1,farm_id,0,70,338,268,0.000000,0.207101,0.792899,10151.973373,4008.835331,2.000000,13378.00,7365.000000,12546.50,12562.000000,numeric,NaN,NaN
2,length,23,69,338,269,0.068047,0.204142,0.795858,46.604698,37.879973,0.600000,253.66,30.000000,36.51,50.000000,numeric,NaN,NaN
3,width,29,76,338,262,0.085799,0.224852,0.775148,29.845825,18.142403,0.450000,190.24,20.000000,30.00,40.000000,numeric,NaN,NaN
4,deep,92,19,338,319,0.272189,0.056213,0.943787,7.133537,26.681219,0.900000,150.00,1.000000,1.30,1.600000,numeric,NaN,NaN
5,created_at,0,142,338,196,0.000000,0.420118,0.579882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-21 02:22:21.000,10
6,updated_at,0,177,338,161,0.000000,0.523669,0.476331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-15 18:16:36.000,10
7,record_id,0,338,338,0,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,ponds_2023_12_21_2117,1
8,extracted_at,0,16,338,322,0.000000,0.047337,0.952663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2024-04-12 17:02:11.000,309
9,max_seed_density,203,94,338,244,0.600592,0.278107,0.721893,248.672850,1077.046893,0.000625,12500.00,63.783784,120.00,185.205992,numeric,NaN,NaN


In [23]:
sam.sample(5)

,cycle_id,updated_at,sampled_at,created_at,average_weight,id,remark
7135,24507.0,2023-09-13 03:11:26,2023-09-12,2023-09-13 03:11:26,6.10,47460.0,NaN
8679,19313.0,2023-02-11 06:00:41,2023-02-11,2023-02-11 06:00:41,17.98,37946.0,NaN
4283,24980.0,2023-10-10 14:17:54,2023-10-10,2023-10-10 14:17:54,6.00,49594.0,NaN
1080,12484.0,2022-03-13 15:14:00,2021-11-17,2022-03-13 15:14:00,3.30,23833.0,NaN
9023,17686.0,2022-10-31 01:19:00,2022-10-30,2022-10-31 01:19:00,12.60,33794.0,NaN


In [24]:
check_health(sam)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2307,15032,12725,0.000000,0.153473,0.846527,19982.632052,5742.703137,3458.00,29874.0,16650.50,20401.00,24690.0000,numeric,NaN,NaN
1,updated_at,0,10515,15032,4517,0.000000,0.699508,0.300492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2021-07-09 23:12:35,45
2,sampled_at,0,1221,15032,13811,0.000000,0.081227,0.918773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-04,109
3,created_at,0,10445,15032,4587,0.000000,0.694851,0.305149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2021-07-09 23:12:35,45
4,average_weight,0,2666,15032,12366,0.000000,0.177355,0.822645,9.900390,7.187137,0.01,100.0,4.49,8.13,13.7725,numeric,NaN,NaN
5,id,0,15016,15032,16,0.000000,0.998936,0.001064,38925.218068,13394.422325,5308.00,61775.0,29964.75,38806.50,49932.2500,numeric,NaN,NaN
6,remark,13693,1150,15032,13882,0.910923,0.076503,0.923497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,#BD-Katalis04,26
